In [8]:
import sys

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

In [5]:
def circular_mean(hours):
    angles = np.array(hours) * (360 / 24)

    sin_angles = np.sin(np.radians(angles))
    cos_angles = np.cos(np.radians(angles))

    mean_sin = np.mean(sin_angles)
    mean_cos = np.mean(cos_angles)

    mean_angle = np.arctan2(mean_sin, mean_cos) * (180 / np.pi)

    if mean_angle < 0:
        mean_angle += 360

    mean_hour = mean_angle * 24 / 360
    
    return mean_hour


In [6]:
df = pd.read_csv('sumup_test.csv')

In [7]:
df

,datetime,"US_300443, UTC-5, -78.1692, 43.0303","US_140906, UTC-6, -95.01667, 38.1","US_155370, UTC-5, -82.38333, 37.53333","US_141673, UTC-6, -95.6355, 37.0613","US_352313, UTC-8, -122.13333, 43.18333","US_419106, UTC-6, -103.8, 31.3","US_157129, UTC-5, -83.0603, 37.7467","US_046232, UTC-8, -120.8991, 39.3675","US_360455, UTC-5, -75.73333, 41.11667",...,"US_485850, UTC-7, -107.71667, 43.43333","US_342994, UTC-6, -95.33333, 35.3","US_034988, UTC-6, -93.5878, 34.5408","US_018209, UTC-6, -86.6708, 32.9205","US_292625, UTC-7, -104.5, 34.13333","US_050105, UTC-7, -104.9504, 37.3169","US_091024, UTC-5, -84.31667, 34.86667","US_232568, UTC-6, -94.0352, 38.868","US_086318, UTC-5, -80.75, 26.56667","US_046577, UTC-8, -119.353, 34.6825"
0,1900010100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1900010101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1900010102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1900010103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1900010104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999309,2013123121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999310,2013123122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999311,2013123123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999312,2014010100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y%m%d%H')
df_season = df[df['datetime'].dt.month.isin([6, 7, 8])]
df_season.iloc[:, 1:] = df_season.iloc[:, 1:].applymap(lambda x: x if x >= 0.1 else np.nan)

In [5]:
def calculate_circular_means(df_season):

    years = range(1900, 2014)
    stations = df_season.columns[1:]  
    result_df = pd.DataFrame(index=years, columns=stations)
    

    df_season['datetime'] = pd.to_datetime(df_season['datetime'])
    df_season['year'] = df_season['datetime'].dt.year
    df_season['hour'] = df_season['datetime'].dt.hour
    
    for station in stations:
        for year in years:

            yearly_data = df_season[df_season['year'] == year]
            

            rain_hours = yearly_data[station].dropna().index.tolist()
            if rain_hours:
                arr_rain_time = yearly_data.loc[rain_hours, 'hour'].tolist()

                mean_hour = circular_mean(arr_rain_time)
                result_df.at[year, station] = mean_hour

    return result_df
df_circular_mean = calculate_circular_means(df_season)
df_circular_mean.to_csv('circular_mean_test.csv', index=False)

In [6]:
df_circular_mean

,"US_300443, UTC-5, -78.1692, 43.0303","US_140906, UTC-6, -95.01667, 38.1","US_155370, UTC-5, -82.38333, 37.53333","US_141673, UTC-6, -95.6355, 37.0613","US_352313, UTC-8, -122.13333, 43.18333","US_419106, UTC-6, -103.8, 31.3","US_157129, UTC-5, -83.0603, 37.7467","US_046232, UTC-8, -120.8991, 39.3675","US_360455, UTC-5, -75.73333, 41.11667","US_354606, UTC-8, -122.7189, 44.6253","US_485850, UTC-7, -107.71667, 43.43333","US_342994, UTC-6, -95.33333, 35.3","US_034988, UTC-6, -93.5878, 34.5408","US_018209, UTC-6, -86.6708, 32.9205","US_292625, UTC-7, -104.5, 34.13333","US_050105, UTC-7, -104.9504, 37.3169","US_091024, UTC-5, -84.31667, 34.86667","US_232568, UTC-6, -94.0352, 38.868","US_086318, UTC-5, -80.75, 26.56667","US_046577, UTC-8, -119.353, 34.6825"
1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009,20.848974,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,9.536104,NaN,NaN,12.783914,16.01328,NaN,NaN,NaN,14.695648,NaN,NaN
2010,8.125532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.36094,NaN,NaN,NaN,15.693155,NaN,NaN,NaN,NaN,NaN,NaN
2011,10.75347,NaN,NaN,NaN,NaN,NaN,NaN,20.565369,NaN,6.831369,NaN,NaN,NaN,16.54079,NaN,NaN,NaN,NaN,NaN,5.0
2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
